In [6]:
import findspark

findspark.init("G:/hadoop/Github/Hadoop-Fundamentals/venv/Lib/site-packages/pyspark")

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
spark = SparkSession.builder.master("local").appName("Linear Regression Model").config("spark.executor.memory", "1gb").getOrCreate()

sc = spark.sparkContext

rdd = sc.textFile('C:/Users/lemon/Downloads/CaliforniaHousing/cal_housing.data')

header = sc.textFile('C:Users/lemon/Downloads/CaliforniaHousing/cal_housing.domain')


rdd = rdd.map(lambda line: line.split(","))

rdd.first()
rdd.take(2)


df = rdd.map(lambda line: Row(longitude=line[0],
                              latitude=line[1],
                              housingMedianAge=line[2],
                              totalRooms=line[3],
                              totalBedRooms=line[4],
                              population=line[5],
                              households=line[6],
                              medianIncome=line[7],
                              medianHouseValue=line[8])).toDF()

df = df.withColumn("longitude", df["longitude"].cast(FloatType()))\
    .withColumn("latitude", df["latitude"].cast(FloatType()))\
    .withColumn("housingMedianAge", df["housingMedianAge"].cast(FloatType()))\
    .withColumn("totalRooms", df["totalRooms"].cast(FloatType()))\
    .withColumn("totalBedRooms", df["totalBedRooms"].cast(FloatType()))\
    .withColumn("population", df["population"].cast(FloatType()))\
    .withColumn("households", df["households"].cast(FloatType()))\
    .withColumn("medianIncome", df["medianIncome"].cast(FloatType()))\
    .withColumn("medianHouseValue", df["medianHouseValue"].cast(FloatType()))



df.show()


+---------+--------+----------------+----------+-------------+----------+----------+------------+----------------+
|longitude|latitude|housingMedianAge|totalRooms|totalBedRooms|population|households|medianIncome|medianHouseValue|
+---------+--------+----------------+----------+-------------+----------+----------+------------+----------------+
|  -122.23|   37.88|            41.0|     880.0|        129.0|     322.0|     126.0|      8.3252|        452600.0|
|  -122.22|   37.86|            21.0|    7099.0|       1106.0|    2401.0|    1138.0|      8.3014|        358500.0|
|  -122.24|   37.85|            52.0|    1467.0|        190.0|     496.0|     177.0|      7.2574|        352100.0|
|  -122.25|   37.85|            52.0|    1274.0|        235.0|     558.0|     219.0|      5.6431|        341300.0|
|  -122.25|   37.85|            52.0|    1627.0|        280.0|     565.0|     259.0|      3.8462|        342200.0|
|  -122.25|   37.85|            52.0|     919.0|        213.0|     413.0|     19